In [4]:
import pandas as pd

## IDF Diabetes Atlas 2021

In [5]:
## MAIN FUNCTION - input
df_input_tc = pd.read_csv('IDF (diabetes-related-health-expenditure-per-person--usd).csv', thousands=',')
country_names = pd.read_csv('../../data/dl1_countrycodeorg_country_name.csv')
df_input_prev = pd.read_csv('./adjust_prevalence/prevalence.csv')

# OUTPUT FILE
save_file_name = "TC_ppp.csv"

In [6]:
## Part I, per patient cost
for i in range(len(df_input_tc)):
    if df_input_tc.loc[i, '2021'] == '-':
        df_input_tc.loc[i, '2021'] = df_input_tc.loc[i, '2011']
        
tc = df_input_tc[['Country/Territory', '2021']]
tc = tc.rename(columns = {'Country/Territory':'country', '2021':'Diabetes mellitus'})
# tc = tc.set_index("countries")
tc.replace({'Taiwan': 'Taiwan (Province of China)', 
            'State of Palestine': 'Palestine', 
            'US Virgin Islands': 'United States Virgin Islands'}, inplace = True)

tc_code = country_names[["Country Code", "country"]].merge(tc, on="country").drop(columns=['country'])
tc_code['Diabetes mellitus'] = tc_code['Diabetes mellitus'].str.replace(',', '')
tc_code['Diabetes mellitus'] = tc_code['Diabetes mellitus'].str.replace('-', '-0').astype(float)

## Part II, per person cost
per_patient = tc_code.set_index("Country Code")
perv = df_input_prev.set_index("Country Code")

data_idf = per_patient * perv / 100000

In [7]:
data_idf = data_idf[data_idf['Diabetes mellitus'] > 0]
data_idf.to_csv(save_file_name)

## JAMA 2016

In [8]:
## MAIN FUNCTION - input

## INPUT FILE
PPP = True
# PPP = False
if PPP:
    df_input_he = pd.read_csv('../health_expenditure/hepc_ppp.csv')
else:
    df_input_he = pd.read_csv('../health_expenditure/hepc.csv')
df_input_prev = pd.read_csv('./adjust_prevalence/prevalence.csv')
df_input_USA_TC = pd.read_csv('TC_USA.csv')

## OUTPUT FILE
# 'tmp.csv'

In [9]:
he = df_input_he[['Country Code', '2013']]
he = he.set_index("Country Code")
he = he.rename(columns = {'2013':'house expend'})
he_merge = he.merge(df_input_prev, on='Country Code')
he_merge = he_merge[["Country Code", "house expend"]]

data = he_merge.set_index("Country Code")
prev = df_input_prev.set_index("Country Code")

In [10]:
for i in range(len(df_input_USA_TC['disease'])):
    disease = df_input_USA_TC.loc[i, 'disease']
    tc_USA = data.loc['USA','house expend'] * df_input_USA_TC.loc[i, 'TC_USA'] / 2100.1
    const = tc_USA / prev.loc['USA', disease] / data.loc['USA','house expend']
    data[disease] = const * prev[disease] * data['house expend']
# data.to_csv(save_file_name)

## Comparison

In [22]:
he2021 = df_input_he[['Country Code', '2021']].set_index("Country Code")['2021']
he2013 = df_input_he[['Country Code', '2013']].set_index("Country Code")['2013']
data_jama_2021 = data
data_jama_2021['Diabetes mellitus'] = data_jama_2021['Diabetes mellitus'] * he2021 / he2013
data_combine = data_idf.merge(data_jama_2021, how='outer', on='Country Code')
data_combine = data_combine.merge(country_names[["Country Code", "Region", "WBCountry"]], on='Country Code')
data_combine = data_combine.set_index(["Region", "WBCountry"])
data_combine = data_combine.sort_index().reset_index()
data_combine.to_csv('tmp.csv',index=False)

,Country Code,Region,WBCountry
0,ABW,Latin America & Caribbean,Aruba
1,AFG,South Asia,Afghanistan
2,AGO,Sub-Saharan Africa,Angola
3,AIA,NaN,NaN
4,ALB,Europe & Central Asia,Albania
...,...,...,...
236,XKX,Europe & Central Asia,Kosovo
237,YEM,Middle East & North Africa,"Yemen, Rep."
238,ZAF,Sub-Saharan Africa,South Africa
239,ZMB,Sub-Saharan Africa,Zambia
